In [1]:
%matplotlib inline               
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('la data total.csv')

In [3]:
df.head()

,Daily Mean PM2.5 Concentration,Max Temp,Avg Dew Point,Max Wind Speed,Avg Humidity
0,8.3,63,25,15,37.0
1,14.3,68,21,7,35.0
2,12.2,57,28,6,47.0
3,22.3,59,42,6,69.0
4,40.3,61,43,7,72.0


In [4]:
df.shape

(3444, 5)

In [5]:
X = df.drop('Daily Mean PM2.5 Concentration',axis=1)
y = df['Daily Mean PM2.5 Concentration']

In [6]:
df = df.reset_index()

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [8]:
X_train.shape

(2755, 4)

In [9]:
X_test.shape

(689, 4)

In [10]:
y_train.shape

(2755,)

In [11]:
y_test.shape

(689,)

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [13]:
X_train.dtypes

Max Temp            int64
Avg Dew Point       int64
Max Wind Speed      int64
Avg Humidity      float64
dtype: object

In [14]:
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [15]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

In [17]:
n=3 # how many times to shuffle the training data
nhn_range=[1, 2, 4, 6, 8, 10] # number of hidden neurons

In [18]:
score_dict = {}
for nhn in nhn_range:
    mlp = MLPRegressor(hidden_layer_sizes=(nhn,), activation='relu', 
                       solver='adam', shuffle=False, random_state=42, 
                       max_iter=2000, momentum=0.7, early_stopping=True, 
                       validation_fraction=0.15)


    nhn_scores = []
    for _ in range(n):

        X_train = shuffle(X_train)
        score = np.sqrt(-cross_val_score(mlp, X_train, 
                    y_train, 
                    cv=5, scoring='neg_mean_squared_error')).mean()
        nhn_scores.append(score)
    score_dict[nhn] = nhn_scores

In [19]:
score_df = pd.DataFrame.from_dict(score_dict)
print(score_df)

         1         2         4         6         8         10
0  7.987905  7.973026  8.052661  8.110872  8.057902  8.034433
1  7.987734  7.979363  8.061267  8.107808  8.066877  8.054547
2  7.987727  7.966574  8.022414  8.101123  8.061644  8.004366


In [26]:
from sklearn.metrics import r2_score

In [27]:
score_dict = {}
for nhn in nhn_range:
    mlp = MLPRegressor(hidden_layer_sizes=(nhn,), activation='relu', 
                       solver='adam', shuffle=False, random_state=42, 
                       max_iter=2000, momentum=0.7, early_stopping=True, 
                       validation_fraction=0.15)


    nhn_scores = []
    for _ in range(n):

        X_train = shuffle(X_train)
        mlp.fit(X_train,y_train)
        predictions = mlp.predict(X_train)
        score = r2_score(y_train , predictions)
        nhn_scores.append(score)
    score_dict[nhn] = nhn_scores

In [28]:
score_df = pd.DataFrame.from_dict(score_dict)
print(score_df)

         1         2         4         6         8         10
0 -0.008862 -0.005253 -0.010667 -0.008106 -0.005008 -0.002595
1 -0.008862 -0.001174 -0.008725 -0.010790 -0.005666  0.001621
2 -0.008861 -0.001304 -0.024967 -0.002049 -0.003586  0.006695
